In [289]:
import copy
import itertools
import numpy as np

In [337]:
class Player(object):
    def __init__(self):
        self.hand = []
        self.usable_ace = 0
        self.showing = 0
        self.reward = 0
        

class BlackJack(object):
    def __init__(self, N=10000):
        self.deck = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10]
        self.hands = list(itertools.product(self.deck[:10], self.deck[:10]))
        self.returns = {0: [[0] * 10] * 10,
                       1: [[0] * 10] * 10}
        self.score = {0: [[0] * 10] * 10,
                       1: [[0] * 10] * 10}
        self.N = N
             
    def reset(self):
        self.dealer = Player()
        self.player = Player()
        
    def draw_card(self):
        """Deals one card from the deck with replacement."""
        return np.random.choice(self.deck)
    
    def draw_hand(self):
        """Deals hand."""
        return [self.draw_card(), self.draw_card()]
    
    def sum_hand(self, hand):
        if self.is_usable(hand): 
            return sum(hand) + 10
        return sum(hand)
    
    def is_natural(self, hand):
        return sorted(hand) == [1, 10]
    
    def is_usable(self, hand):
        return 1 in hand and sum(hand) + 10 <= 21
        
    def is_bust(self, hand):
        if self.sum_hand(hand) > 21:
            return True
        
    def play_hand(self, hand=None):
        # dealer cards
        self.dealer.hand = self.draw_hand()
        
        # player cards
        if hand is not None:
            self.player.hand = copy.copy(hand)
        else:
            self.player.hand = self.draw_hand()
            
        self.player.showing = self.dealer.hand[1]
                
        # draw player cards 
        while self.sum_hand(self.player.hand) < 20:
            self.player.hand.append(self.draw_card())      
        
        # draw dealer cards
        while self.sum_hand(self.dealer.hand) < 17:
            self.dealer.hand.append(self.draw_card())      
               
        # check if player has usable ace
        if self.is_usable(self.player.hand):
            self.player.usable_ace = 1
            
        # calculate reward
        if self.is_bust(self.player.hand):
            self.player.reward = -1
            
        elif self.is_bust(self.dealer.hand):
            self.player.reward = 1
            
        elif self.sum_hand(self.player.hand) < self.sum_hand(self.dealer.hand):
            self.player.reward = -1
            
        elif self.sum_hand(self.player.hand) == self.sum_hand(self.dealer.hand):
            self.player.reward = 0
            
        elif self.sum_hand(self.player.hand) > self.sum_hand(self.dealer.hand):
            self.player.reward = 1
            
        return self.sum_hand(self.player.hand), self.player.usable_ace, self.player.showing, self.player.reward
            
    def simulate(self):
        for hand in self.hands:
            hand = list(hand)
            rewards = [[0] * 31] * 10
            for i in range(self.N):
                self.reset()
                score, has_ace, showing, reward = self.play_hand(hand)
                rewards[showing-1][score] +=reward
#                 print(score-12)
#                 if score <= 21:
#                     self.returns[has_ace][showing-1][score-12] += reward
#             print(sum(rewards) / self.N)
        print(rewards)

In [339]:
blackjack = BlackJack(1000)
blackjack.simulate()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
